In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('per.csv')
data.head()

,NewsID,Title,Body,Date,Time,Category,Category2
0,843656,\nوزير علوم درجمع استادان نمونه: سن بازنشستگي ...,\nوزير علوم در جمع استادان نمونه كشور گفت: از ...,\n138/5//09,\n0:9::18,\nآموزشي-,\nآموزشي
1,837144,\nگردهمايي دانش‌آموختگان موسسه آموزش عالي سوره...,\nبه گزارش سرويس صنفي آموزشي خبرگزاري دانشجويا...,\n138/5//09,\n1:4::11,\nآموزشي-,\nآموزشي
2,436862,\nنتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور...,\nنتايج آزمون دوره‌هاي فراگير مقاطع كارشناسي و...,\n138/3//07,\n1:0::03,\nآموزشي-,\nآموزشي
3,227781,\nهمايش يكروزه آسيب شناسي مفهوم روابط عمومي در...,\n,\n138/2//02,\n1:3::42,\nاجتماعي-خانواده-,\nاجتماعي
4,174187,\nوضعيت اقتصادي و ميزان تحصيلات والدين از مهمت...,\nمحمدتقي علوي يزدي، مجري اين طرح پژوهشي در اي...,\n138/1//08,\n1:1::49,\nآموزشي-,\nآموزشي


In [3]:
with open('stopwords.txt') as stopwords_file :
    stopwords = stopwords_file.readlines()
stopwords = [line.replace('\n','') for line in stopwords]
len(stopwords)

1316

In [4]:
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\MILAD-
[nltk_data]     CO\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [5]:
nltk_stopwords = nltk.corpus.stopwords.words('english')
stopwords.extend(nltk_stopwords)
len(stopwords)

1495

In [6]:
import hazm

In [7]:
stemmer = hazm.Stemmer()

In [8]:
stemmer.stem('استادها')

'استاد'

In [9]:
lemmatizer = hazm.Lemmatizer()
lemmatizer.lemmatize('می‌روم')

'رفت#رو'

In [10]:
from hazm import word_tokenize

In [11]:
dataset = pd.DataFrame(columns=('title_body','category'))
for index, row in data.iterrows():
    title_body = row['Title'] + ' ' + row['Body']
    title_body_tokenized = word_tokenize(title_body)
    title_body_tokenized_filtered = [w for w in title_body_tokenized if not w in stopwords]
    title_body_tokenized_filtered_stemmed = [stemmer.stem(w) for w in title_body_tokenized_filtered]
    title_body_tokenized_filtered_lemmatized = [lemmatizer.lemmatize(w).replace('#',' ') for w in title_body_tokenized_filtered]
    dataset.loc[index]= {
        'title_body' : ' '.join(title_body_tokenized_filtered_lemmatized)+' '+ ' '.join(title_body_tokenized_filtered_stemmed),
        'category' : row['Category2'].replace('\n','')
    }

In [12]:
dataset.head()

,title_body,category
0,وزير علوم درجمع استاد نمونه سن بازنشستگي استاد...,آموزشي
1,گردهمايي دانش‌آموختگان موسسه آموزش عالي سوره ب...,آموزشي
2,نتايج آزمون دوره‌هاي فراگير دانشگاه پيام‌نور ا...,آموزشي
3,همايش يكروزه آسيب شناسي مفهوم روابط عمومي در ب...,اجتماعي
4,وضعيت اقتصادي و ميزان تحصيلات والدين از مهمتري...,آموزشي


In [13]:
from sklearn.feature_extraction.text import TfidfVectorizer

In [14]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
vectorizer.fit(dataset.title_body)

TfidfVectorizer(ngram_range=(1, 2))

In [15]:
import pickle
with open('lion_v.jsdh','wb') as f:
    pickle.dump(vectorizer,f)

In [16]:
X = vectorizer.transform(dataset.title_body)

In [17]:
X

<10999x1754943 sparse matrix of type '<class 'numpy.float64'>'
	with 8000883 stored elements in Compressed Sparse Row format>

In [18]:
from sklearn.preprocessing import LabelEncoder

In [19]:
le = LabelEncoder()
Y = le.fit_transform(dataset.category)
Y

array([ 0,  0,  0, ..., 10, 10, 10])

In [20]:
with open('lion_le.jsdh','wb') as f:
    pickle.dump(le,f)

In [21]:
import numpy as np

In [22]:
np.unique(dataset.category)

array(['آموزشي', 'اجتماعي', 'اقتصادي', 'بهداشتي', 'تاريخي', 'سياسي',
       'علمي', 'فرهنگي', 'فقه و حقوق', 'مذهبي', 'ورزشي'], dtype=object)

In [23]:
np.unique(Y)

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10])

In [24]:
len(Y)

10999

In [25]:
from sklearn.model_selection import train_test_split

In [26]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)

In [27]:
from sklearn import svm

In [28]:
svc = svm.SVC()
svc.fit(X_train,y_train)
svc.score(X_test,y_test)

0.8429090909090909

In [29]:
with open('lion_svc.jsdh','wb') as f:
    pickle.dump(svc,f)

In [30]:
from sklearn.metrics import classification_report,confusion_matrix

In [31]:
y_pred = svc.predict(X_test)
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.84      0.92      0.88       252
           1       0.64      0.68      0.66       258
           2       0.82      0.81      0.82       242
           3       0.84      0.95      0.89       243
           4       0.92      0.88      0.90       263
           5       0.76      0.74      0.75       227
           6       0.80      0.73      0.76       261
           7       0.82      0.90      0.86       232
           8       0.91      0.87      0.89       260
           9       0.97      0.87      0.92       268
          10       0.97      0.92      0.95       244

    accuracy                           0.84      2750
   macro avg       0.85      0.84      0.84      2750
weighted avg       0.85      0.84      0.84      2750



In [32]:
print(confusion_matrix(y_test,y_pred))

[[232   5   1   0   0   2  11   1   0   0   0]
 [ 13 176  15   9   3  10  12  12   5   2   1]
 [  2  20 197   1   0  10   9   3   0   0   0]
 [  2   2   1 230   0   2   6   0   0   0   0]
 [  2   8   1   0 232   9   2   6   2   1   0]
 [  3  15   8   1  10 168   2   6  12   1   1]
 [ 16   6   6  32   1   2 190   6   1   0   1]
 [  3   8   1   0   3   1   4 209   0   2   1]
 [  2   9   4   0   2  11   0   3 227   0   2]
 [  0  19   4   2   1   3   1   5   1 232   0]
 [  1   7   2   0   0   3   1   4   1   0 225]]


In [34]:
import os
os.getcwd()


'E:\\movie\\آموزشی\\مهندسی داده\\7learn\\part1\\PROJECT\\7learn\\application\\news'

In [37]:
dir_path = os.path.dirname(os.path.realpath(__file__))
dir_path

NameError: name '__file__' is not defined